In [ ]:
import pandas as pd
import os
import string
import re
import numpy as np 

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
#dummy function

def cal_sum(a,b):
    return a+b

In [ ]:
def lower_target_variable(x):
    if isinstance(x, str):
        return x.lower()
    else:
        return x    

def create_unique_username(prefix, x):
    x = prefix+ str(x)
    return x
    
def output_cleaned_individual_csv(temp_df_a, file_name):
    csv_op_folder_path = '/content/drive/MyDrive/CDAC-SELF/CDAC_PROJECT/Mental_Graph_Of_User_Using_Social_Media_Posts/project_code/files/cleaned/'
    
    split_fname = os.path.splitext(file_name)
    temp_path = split_fname[0]+'_cleaned'+split_fname[1]

    op_path = os.path.join(csv_op_folder_path, temp_path)

    ######
    temp_df_a = drop_duplicates(temp_df_a)
    temp_df_a = drop_na_values_category(temp_df_a)
    temp_df_a = cleaning(temp_df_a)
    ######
    temp_df_a.to_csv(op_path, index = False, header=True)


    

In [ ]:
def concat_csvs(csv_folder_path):

    dataframes = []
    for file in os.listdir(csv_folder_path):
        if file.endswith('.csv'):
            temp_path = os.path.join(csv_folder_path, file)

            temp_df = pd.read_csv(temp_path)
            
            #1. Sanity Check
            #providing neutral name to tweet/comment column
            if(file.startswith('tweets')):
                temp_df.rename(columns={'Tweet':'orig_sentence'}, inplace = True)
                # temp_df.rename(columns={'username':'unique_username'}, inplace = True)
                temp_df['username'] = temp_df['username'].apply(lambda x: create_unique_username('tweets_',x) )

            elif(file.startswith('ytComments')):
                temp_df.rename(columns={'textOriginal':'orig_sentence'}, inplace = True)
                temp_df.rename(columns={'authorDisplayName':'username'}, inplace = True)
                temp_df['username'] = temp_df['username'].apply(lambda x: create_unique_username('ytComments_',x) )

            else:
                print("Unformatted column name found in file:", file)  

            #2. Sanity Check
            #checking both target variables is available or not
            if 'category' not in temp_df.columns:
                print('"category" column not found in file:', file)
            elif 'sub-category' not in temp_df.columns:
                print('"sub-category" column not found in file:', file)


            temp_df = temp_df[['username', 'orig_sentence', 'category', 'sub-category']]
            

            #3. Sanity Check
            #lowering target variables
            temp_df['category'] = temp_df['category'].apply(lambda x: lower_target_variable(x) )
            temp_df['sub-category'] = temp_df['sub-category'].apply(lambda x: lower_target_variable(x))

            ###
            # print("Filename",file,"Category", temp_df['category'].unique())
            # print("Filename",file,"Category", temp_df['sub-category'].unique())

            ###
            
            ###converting neutral to positive-starts
            temp_df.loc[temp_df["category"] == "neutral", "category"] = "positive"
            ###converting neutral to positive-ends

            #output cleaned individual csv
            dataframes.append(temp_df)
            # output_cleaned_individual_csv(temp_df, file)
            

    df = pd.concat(dataframes)
    return df 


In [ ]:

def drop_duplicates(df):
    df.drop_duplicates() #print it
    return df

def drop_na_values_category(df):
    df.dropna(subset=['category'], inplace=True) #print it
    return df

def drop_na_values_subcategory(subc_df):
    print('drop_na_values_subcategory:', subc_df.shape)
    subc_df = subc_df.dropna(subset=['sub-category']) #print it
    print('drop_na_values_subcategory:', subc_df.shape)


    # drop_subcategory.dropna(subset=['sub-category'], inplace=True) #print it

    return subc_df




def output_merged_cleaned_csv(df):
    df.to_csv(r'/content/drive/MyDrive/CDAC-SELF/CDAC_PROJECT/Mental_Graph_Of_User_Using_Social_Media_Posts/project_code/files/cleaned/merged_cleaned.csv', index = False, header=True)
    return None

In [ ]:
#Cleaning

def remove_newline(text):
    nl = re.compile('\n')
    return nl.sub(r' ', text)

def remove_hastag(text):
    ht = re.compile('#[^\s]+')
    return ht.sub(r'',text)
    

def remove_at_symbol(text):
    at = re.compile('@[^\s]+')
    return at.sub(r'',text)
    

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)
    

def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_html(text):
    html = re.compile(r'^[^ ]<.*?>|&([a-z0-9]+|#[0-9]\"\'\“{1,6}|#x[0-9a-f]{1,6});[^A-Za-z0-9]+')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def remove_quotes(text):
    quotes = re.compile(r'[^A-Za-z0-9\s]+')
    return re.sub(quotes, '', text)



In [ ]:
def applying_helper_functions(df):

    df['mod_text'] = df['orig_sentence'].apply(lambda x: remove_newline(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_hastag(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_at_symbol(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_URL(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_emoji(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_html(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_punct(x))
    df['mod_text'] = df['mod_text'].apply(lambda x: remove_quotes(x))

    return df


In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def stopwords_setup(stopwords):

    stop_words = set(stopwords.words("english"))
    stop_words.remove('not')
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stop_words = stop_words.union(more_stopwords)

    return stop_words

In [ ]:
def select_columns(df, prediction=True):
    if prediction:
        df = df[['username','orig_sentence',"cleaned_sentence"]]
    else:    
        df = df[['username','orig_sentence',"cleaned_sentence", "category", "sub-category"]]
    return df

In [ ]:
# Tokenizing the tweet base texts.

def cleaning(df, prediction=False):

    if prediction:
        df['category'] = None
        df['sub-category'] = None
        #for cleaning part prediction -starts
        if 'Tweet' in df.columns:
            df.rename(columns={'Tweet':'orig_sentence'}, inplace = True)
            print('column renamed: Tweet orig_sentence')

        if 'textOriginal' in df.columns:
            df.rename(columns={'textOriginal':'orig_sentence'}, inplace = True)
            print('columns renamed textOriginal orig_sentence')
        #for cleaning part prediction -ends

    df = applying_helper_functions(df)

    stop_words = stopwords_setup(stopwords)
    df['tokenized'] = df['mod_text'].apply(word_tokenize)
    df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])
    df['stopwords_removed'] = df['lower'].apply(lambda x: [word for word in x if word not in stop_words])
    df['pos_tags'] = df['stopwords_removed'].apply(nltk.tag.pos_tag)
    df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

    wnl = WordNetLemmatizer()

    df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
    df['lemmatized'] = df['lemmatized'].apply(lambda x: [word for word in x if word not in stop_words])
    df['cleaned_sentence'] = [' '.join(map(str, l)) for l in df['lemmatized']]

    
    df = select_columns(df ,prediction=False)
    # print(df.shape)
    df['cleaned_sentence'].replace('', np.nan, inplace=True)
    df.dropna(subset=['cleaned_sentence'], inplace=True)
    # print(df.shape)

    return df
# df.head()

In [ ]:
def splitting_train_test(df,TRAIN_SIZE):
    df_train, df_test = train_test_split(df, test_size=1-TRAIN_SIZE, random_state=42)
    print("TRAIN size:", len(df_train))
    print("TEST size:", len(df_test))

    return df_train, df_test

In [ ]:
def labelEncoding_target_variables(df_train, df_test):
    encoder = LabelEncoder()
    encoder.fit(df_train.category.tolist())

    y_train = encoder.transform(df_train.category.tolist())
    y_test = encoder.transform(df_test.category.tolist())

    y_train = y_train.reshape(-1,1)
    y_test = y_test.reshape(-1,1)

    print("y_train",y_train.shape)
    print("y_test",y_test.shape)

    return encoder, y_train, y_test

In [ ]:
def tokenizing(df_train):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df_train.cleaned_sentence)

    vocab_size = len(tokenizer.word_index) + 1
    # print("Total words", vocab_size)

    return tokenizer

def converting_txt2seq(tokenizer, df_train, df_test, SEQUENCE_LENGTH):
    x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.cleaned_sentence), maxlen=SEQUENCE_LENGTH)
    x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.cleaned_sentence), maxlen=SEQUENCE_LENGTH)

    return x_train, x_test

In [ ]:
###### WORD2VEC 

In [ ]:
# Parameters for WORD2VEC
W2V_SIZE = 50
W2V_WINDOW = 7
W2V_EPOCH = 50
W2V_MIN_COUNT = 3

In [ ]:
from gensim.models import Word2Vec

def create_Word2Vec_embedding(df_train):
    
    documents = [text.split() for text in df_train.cleaned_sentence] 
    w2v_model = Word2Vec(size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=8)
    w2v_model.build_vocab(documents)
    w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

    w2v_weights = w2v_model.wv.vectors
    w2v_vocab_size, w2v_embedding_size = w2v_weights.shape

    return w2v_model, w2v_weights, w2v_vocab_size, w2v_embedding_size


In [ ]:
def create_embeddingMatrix_from_Word2Vec(w2v_model, vocab_size, W2V_SIZE, tokenizer):

    embedding_matrix = np.zeros((vocab_size, W2V_SIZE))
    for word, i in tokenizer.word_index.items():
        if word in w2v_model.wv:
            embedding_matrix[i] = w2v_model.wv[word]
        
    return embedding_matrix